# Quantum music

NOTE: this notebook follows the contents of chapter 08 of **Mastering Quantum Computing with IBM QX**. The original code is available at https://github.com/PacktPublishing/Mastering-Quantum-Computing-with-IBM-QX

In [1]:
%matplotlib inline
from qiskit import Aer
from qiskit import IBMQ

backend = Aer.get_backend('qasm_simulator')

In [2]:
import numpy as np
import pygame, pygame.sndarray
import json


# read midi conversion table
with open("midi.json") as f:
        midi_conversion_tables = json.load(f)
    
midi_to_note_bin = midi_conversion_tables['midi_to_note_bin']
midi_to_frequency_bin = midi_conversion_tables['midi_to_frequency_bin']


def play_notes(freqs, volumes, duration=1000):
    """
    freqs: a list of frequencies (Hz)
    volumes: a list of volumes in the range [0..1]
    duration: in milliseconds
    
    example usage:
    play_notes([440,880],[0.6,0.2])
    """
    
    def wave(hz, volume):
        amp = (2**14) * volume # 14 bit volume depth
        n = int(44100/float(hz)) # num of samples
        return amp * np.sin(np.arange(n) * np.pi * 2 / n)
    
    pygame.mixer.init()
    n_samples = int(44100 * duration / 1000)
    sample_wave = sum(np.resize(wave(hz, volume), (n_samples,)).astype(np.int16)  for hz, volume in zip(freqs, volumes))
    stereo = np.vstack((sample_wave, sample_wave)).T.copy(order='C')
    
    sound = pygame.sndarray.make_sound(stereo)
    
    sound.play(-1)
    pygame.time.delay(duration)
    sound.stop()
    pygame.time.delay(duration)
    

def play_all():
    for k,v in midi_conversion_tables["note_to_frequency"].items():
        print(k, end=' ')
        play_notes([float(v)], [0.8], duration=400)
        

play_all()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
A0 A#0 B0 C1 C#1 D1 D#1 E1 F1 F#1 G1 G#1 A1 A#1 B1 C2 C#2 D2 D#2 E2 F2 F#2 G2 G#2 A2 A#2 B2 C3 C#3 D3 D#3 E3 F3 F#3 G3 G#3 A3 A#3 B3 C4 C#4 D4 D#4 E4 F4 F#4 G4 G#4 A4 A#4 B4 C5 C#5 D5 D#5 E5 F5 F#5 G5 G#5 A5 A#5 B5 C6 C#6 D6 D#6 E6 F6 F#6 G6 G#6 A6 A#6 B6 C7 C#7 D7 D#7 E7 F7 F#7 G7 G#7 A7 A#7 B7 C8 

In [3]:
def quantum_play_notes(qc, shots, backend=None):
    """
    The quantum circuit 'qc' is executed 'shots' times. 
    Each time the measured output is converted using the 
    midi standard and played.
    """
    backend = backend or Aer.get_backend('qasm_simulator')
    
    for i in range(shots):
        sim = qiskit.execute(qc, backend=backend, shots=1)
        result = sim.result().get_counts(qc)
        # convert the measured values to note and frequencies
        
        for k in result.keys():
            print(midi_to_note_bin[k], end=' ')   
            play_notes([float(midi_to_frequency_bin[k])], [1.0])
        
        print()


def quantum_play_chords(qc, shots, backend=None):
    """
    Play a chord.
    A quantum circuit 'qc' is executed and measured 'shots' times.
    Then the results are collected and mapped to a superposition
    of notes.
    """
    backend = backend or Aer.get_backend('qasm_simulator')
    sim = qiskit.execute(qc, backend=backend, shots=shots)
    results = sim.result().get_counts(qc).items()
    
    freqs = []
    volumes = []
    
    for k, v in results:
        freqs += [float(midi_to_frequency_bin[k])]
        volumes += [int(v)/shots]
        
        if k in midi_to_note_bin:
            print('%f percent' % (int(v) / shots * 100), midi_to_note_bin[k])
        else:
            print('%f percent' % (int(v) / shots * 100), k)

    play_notes(freqs,volumes)

In [4]:
import qiskit
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit, execute

qr = QuantumRegister(7)
cr = ClassicalRegister(7)
qc = QuantumCircuit(qr, cr)

qc.x(qr[4])
qc.x(qr[5])
qc.h(qr[2])
qc.h(qr[3])

qc.measure(qr, cr)

quantum_play_notes(qc, 20)
quantum_play_chords(qc, 20)

G#3 
E3 
G#3 
C4 
C4 
G#3 
C4 
G#3 
C3 
C3 
C4 
C4 
E3 
E3 
G#3 
C4 
E3 
C3 
C3 
E3 
35.000000 percent C3
20.000000 percent G#3
25.000000 percent E3
20.000000 percent C4
